# Import Libraries

In [1]:
import gym
import numpy as np
import time
from IPython import display

# Value Iteration

In [4]:
def value_iteration(env, max_iters=1000, gamma=0.9):
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Compute the value for state
        for state in range(env.observation_space.n):
            q_values = []
            # Compute the q-value for each action
            for action in range(env.action_space.n):
                q_value = 0
                # Loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # Select the best action
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            # Converged iteration at time complexity O(|S|) 
            print(f'Converged at {(i+2)*env.action_space.n}-th iteration.')
            break
    
    return v_values

In [3]:
def policy_extraction(env, v_values, gamma=0.9):
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # Compute the best action for each state in the game
    # Compute q-value for each (state-action) pair in the game
    for state in range(env.observation_space.n):
        q_values = []
        # Compute q_value for each action
        for action in range(env.action_space.n):
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            q_values.append(q_value)
        
        # Select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

# Policy Iteration

In [5]:
def policy_evaluation(env, v_values, policy, max_iters=1000, gamma=0.9):
    for i in range(max_iters):
        prev_v_values = np.copy(v_values)
        for state in range(env.observation_space.n):
            v_values[state] = 0
            for prob, next_state, reward, done in env.P[state][policy[state]]:
                v_values[state] += prob * (reward + gamma * prev_v_values[next_state])

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            break

    return i+1, v_values

In [6]:
def policy_improvement(env, v_values, policy, gamma=0.9):
    for state in range(env.observation_space.n):
        q_values = []
        # Compute Q_value for each action
        for action in range(env.action_space.n):
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # Selecting action haved best Q_values
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [7]:
def policy_iteration(env, max_loops=1000, max_eva_iters=1000, gamma=0.9):
    v_values = np.zeros(env.observation_space.n)
    policy = np.zeros(env.observation_space.n, dtype=np.int)
    iterations = 0

    for _ in range(max_loops):
        iters, v_values = policy_evaluation(env, v_values, policy, max_eva_iters, gamma)
        prev_policy, policy = np.copy(policy), policy_improvement(env, v_values, policy, gamma)

        iterations += iters + env.action_space.n

        if np.all(np.isclose(policy, prev_policy)):
            # Converged iteration at time complexity O(|S|)
            print(f'Converged at {iterations}-th iterations.')
            break

    return policy

# Running

In [8]:
def play(env, policy):
    state = env.reset()
    total_reward = 0
    done = False
    steps = 0
    #time.sleep(1)
    #display.clear_output(wait=True)
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        #print(f'Step {steps}')
        #env.render()
        #time.sleep(0.2)
        #if not done:
        #    display.clear_output(wait=True)
        state = next_state

    return total_reward

In [9]:
def play_multiple_times(env, policy, max_episodes=1000):
    reward = []
    success = 0

    for i in range(max_episodes):
        reward.append(play(env, policy))

        if reward[i] > 0:
            success += 1
    
    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Reward-average: {np.average(reward)}')

## FrozenLake-v0

In [10]:
env1 = gym.make('FrozenLake-v0')

In [19]:
''' Value Iteration '''
start = time.time()
v_values_00 = value_iteration(env1, max_iters=1000, gamma=0.9)
policy_00 = policy_extraction(env1, v_values_00, gamma=0.9)
print(f'Runtime: {time.time() - start}')
play_multiple_times(env1, policy_00, 1000)

Converged at 324-th iteration.
Runtime: 0.06965208053588867
Number of successes: 725/1000
Reward-average: 0.725


In [20]:
''' Policy Iteration '''
start = time.time()
policy_01 = policy_iteration(env1)
print(f'Runtime: {time.time() - start}')
play_multiple_times(env1, policy_01, 1000)

Converged at 256-th iterations.
Runtime: 0.0457158088684082
Number of successes: 732/1000
Reward-average: 0.732


In [13]:
''' Policy comparation '''
#print(policy_00)
#print(policy_01)
print(np.where(policy_00 != policy_01))

(array([], dtype=int64),)


## FrozenLake8x8-v0

In [14]:
env2 = gym.make('FrozenLake8x8-v0')

In [38]:
''' Value Iteration '''
start = time.time()
v_values_10 = value_iteration(env2, max_iters=1000, gamma=0.9)
policy_10 = policy_extraction(env2, v_values_10, gamma=0.9)
print(f'Runtime: {time.time() - start}')
play_multiple_times(env2, policy_10, 1000)

Converged at 476-th iteration.
Runtime: 0.16405510902404785
Number of successes: 738/1000
Reward-average: 0.738


In [39]:
''' Policy Iteration '''
start = time.time()
policy_11 = policy_iteration(env2)
print(f'Runtime: {time.time() - start}')
play_multiple_times(env2, policy_11, 1000)

Converged at 544-th iterations.
Runtime: 0.19114303588867188
Number of successes: 730/1000
Reward-average: 0.73


In [27]:
''' Policy comparation '''
#print(policy_10)
#print(policy_11)
print(np.where(policy_10 != policy_11))

(array([], dtype=int64),)


## Taxi-v3

In [28]:
env3 = gym.make('Taxi-v3')

In [31]:
''' Value Iteration '''
start = time.time()
v_values_20 = value_iteration(env3, max_iters=1000, gamma=0.9)
policy_20 = policy_extraction(env3, v_values_20, gamma=0.9)
print(f'Runtime: {time.time() - start}')
play_multiple_times(env3, policy_20, 1000)

Converged at 708-th iteration.
Runtime: 1.393510341644287
Number of successes: 1000/1000
Reward-average: 8.016


In [32]:
''' Policy Iteration '''
start = time.time()
policy_21 = policy_iteration(env3)
print(f'Runtime: {time.time() - start}')
play_multiple_times(env3, policy_21, 1000)

Converged at 358-th iterations.
Runtime: 0.7198879718780518
Number of successes: 1000/1000
Reward-average: 7.906


In [33]:
''' Policy comparation '''
#print(policy_20)
#print(policy_21)
print(np.where(policy_20 != policy_21))

(array([], dtype=int64),)
